In [1]:
import ROOT
import numpy as np
from itertools import combinations_with_replacement
from ast import literal_eval

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x9ee6890


In [2]:
input_path = "fit-configs/signal-only/fit-inputs-0.0-0.0-reco_oo.txt"
# input_path = "fit-configs/signal-only/fit-inputs-0.0-0.0-mc_oo.txt"
# input_path = "fit-configs/signal-only/fit-inputs-0.0-0.0-av_mc_oo.txt"

In [3]:
with open(input_path) as infile:
    lines = infile.readlines()
n_per_ab = literal_eval(lines[1])
nominal_means = literal_eval(lines[2])
slopes = literal_eval(lines[3])
Cov_O = literal_eval(lines[4])
print(lines)

['#evt/ab_inv, means, slopes, cov\n', '2022377.202575377\n', '[0.025102206287087838, 0.04795018914790096, 0.0647069553362756]\n', '[[16.453997746930632, 3.6333378331753816, -9.742560084453636], [1.011958347460184, 3.7941217592848138, 0.008583486433210117], [-5.051637323324201, -0.6220281135795896, 9.088317890899981]]\n', '[[0.526004905526162, 0.11740744424050836, -0.2590917059455628], [0.11740744424050836, 0.3730411566425547, 0.024186149615180685], [-0.2590917059455628, 0.024186149615180685, 1.033176508820112]]\n']


In [4]:
n_obs = len(nominal_means)
run_ab = 3

In [5]:
coupling_names = ["g1z", "ka", "lz"]

In [6]:
Cov_mean_O = np.asarray(Cov_O) / (n_per_ab * run_ab)

In [7]:
couplings = []
for i in range(3):
    cpl = ROOT.RooRealVar(coupling_names[i], coupling_names[i], 0., -0.5, 0.5)
    couplings.append(cpl)

obs = []
mus_exp = []
all_vars = []
rel_changes = []
for i, m in enumerate(nominal_means):
    sigma = np.sqrt(Cov_mean_O[i,i])
    o = ROOT.RooRealVar(f"O_{i+1}", f"O_{i+1}", m, m-5*sigma, m+5*sigma)
    obs.append(o)

    vars = []
    for j in range(3):
        # var = ROOT.RooProduct(f"var_{i}_{j}", f"var_{i}_{j}", couplings[j], ROOT.RooFit.RooConst(slopes[j][i]))
        var = ROOT.RooProduct(f"var_{i}_{j}", f"var_{i}_{j}", couplings[j], ROOT.RooFit.RooConst(slopes[i][j]))
        vars.append(var)
    all_vars += vars
    nominal_exp = ROOT.RooFit.RooConst(m)
    # mu_exp = ROOT.RooAddition(f"mu_exp_{i}", f"mu_exp_{i}", [nominal_exp] + vars)
    rel_change = ROOT.RooAddition(f"rel_change_{i}", f"rel_change_{i}", [ROOT.RooFit.RooConst(1.0)] + vars)
    rel_changes.append(rel_change)
    mu_exp = ROOT.RooProduct(f"mu_exp_{i}", f"mu_exp_{i}", rel_change, nominal_exp)
    mus_exp.append(mu_exp)

print(all_vars)

Cov_mean_O_root = ROOT.TMatrixDSym(n_obs)
for i, j in combinations_with_replacement(range(3), r=2):
    Cov_mean_O_root[i][j] = Cov_mean_O[i, j]
    Cov_mean_O_root[j][i] = Cov_mean_O[i, j]

model = ROOT.RooMultiVarGaussian("model", "model", obs, mus_exp, Cov_mean_O_root)

[<cppyy.gbl.RooProduct object at 0xd0c5c90>, <cppyy.gbl.RooProduct object at 0xd023a00>, <cppyy.gbl.RooProduct object at 0xd0d1b80>, <cppyy.gbl.RooProduct object at 0xd3fa710>, <cppyy.gbl.RooProduct object at 0xd40b740>, <cppyy.gbl.RooProduct object at 0xd2f78a0>, <cppyy.gbl.RooProduct object at 0xd357d40>, <cppyy.gbl.RooProduct object at 0x9f77f00>, <cppyy.gbl.RooProduct object at 0xd339310>]


In [8]:
model.Print("t")

0xd6c66e0 RooMultiVarGaussian::model = 1 [Auto,Dirty] 
  0xaa4e2f0/V- RooRealVar::O_1 = 0.0251022
  0xd26ad90/V- RooRealVar::O_2 = 0.0479502
  0x6744960/V- RooRealVar::O_3 = 0.064707
  0xd2874a0/V- RooProduct::mu_exp_0 = 0.0251022 [Auto,Clean] 
    0xd33b580/V- RooAddition::rel_change_0 = 1 [Auto,Clean] 
      0xd0ecc50/V- RooConstVar::1 = 1
      0xd0c5c90/V- RooProduct::var_0_0 = 0 [Auto,Clean] 
        0xce30610/V- RooRealVar::g1z = 0
        0xd011400/V- RooConstVar::16.454 = 16.454
      0xd023a00/V- RooProduct::var_0_1 = 0 [Auto,Clean] 
        0xcd33270/V- RooRealVar::ka = 0
        0xcf40700/V- RooConstVar::3.63334 = 3.63334
      0xd0d1b80/V- RooProduct::var_0_2 = 0 [Auto,Clean] 
        0xca969a0/V- RooRealVar::lz = 0
        0xd0eb5d0/V- RooConstVar::-9.74256 = -9.74256
    0xd0ed6c0/V- RooConstVar::0.0251022 = 0.0251022
  0xd236cf0/V- RooProduct::mu_exp_1 = 0.0479502 [Auto,Clean] 
    0xd3b3f00/V- RooAddition::rel_change_1 = 1 [Auto,Clean] 
      0xd0ecc50/V- RooConstVar::1

In [9]:
w = ROOT.RooWorkspace("w", "workspace")
# w.Import(model)
modelconfig = ROOT.RooStats.ModelConfig("modelconfig", w)
modelconfig.SetPdf(model)
modelconfig.SetParametersOfInterest(couplings)
modelconfig.SetObservables(obs)
# w.Print()
w.Import(modelconfig)
w.writeToFile("gen-lvl-standalone-ws.root")
w

In [10]:
ds = ROOT.RooStats.AsymptoticCalculator.GenerateAsimovData(model, obs)
ds.Print("v")

DataStore CountingAsimovData0 (CountingAsimovData0)
  Contains 1 entries
  Observables: 
    1)  O_1 = 0.0251022  L(0.02363 - 0.0265744)  "O_1"
    2)  O_2 = 0.0479502  L(0.0467104 - 0.04919)  "O_2"
    3)  O_3 = 0.064707  L(0.0626436 - 0.0667703)  "O_3"


In [11]:
nll = model.createNLL(ds)

[#1] INFO:Fitting -- RooAbsPdf::fitTo(model_over_model_Int[O_1,O_2,O_3]) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- using generic CPU library compiled with no vectorizations
[#1] INFO:Fitting -- Creation of NLL object took 8.9078 ms


In [12]:
nll_minimizer = ROOT.RooMinimizer(nll)

[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_over_model_Int[O_1,O_2,O_3]_CountingAsimovData0) Summation contains a RooNLLVar, using its error level


In [13]:
nll_minimizer.migrad()

0

Minuit2Minimizer: Minimize with max-calls 1500 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = -21.5818820435083687
Edm   = 0
Nfcn  = 29
g1z	  = 0	 +/-  0.00105608	(limited)
ka	  = 0	 +/-  0.00140882	(limited)
lz	  = 0	 +/-  0.00100514	(limited)


Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 135.603 μs
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -21.58188204 Edm =                 0 NCalls =     13
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : -21.58188204
  Edm           : 0
  Internal parameters:	[                0                0                0]	
  Internal gradient  :	[                0                0                0]	
  Internal covariance matrix:
[[  3.8302285e-06              0              0]
 [              0  1.4278039e-05              0]
 [              0              0  3.7075639e-06]]]
Info in <Minuit2>: VariableMetricBuilder Start iterating until Edm is < 0.001 with call limit = 1500
Info in <Minuit2>: VariableMetricBuilder    0 - FCN =      -21.58188204 Edm =                 0 NCalls =     13
Info in <Minuit2>: MnHesse Done after 45.239 μs
Info in <Minuit2>: Var

In [14]:
nll_minimizer.minos(couplings)

0

******************************************************************************************************
Minuit2Minimizer::GetMinosError - Run MINOS LOWER error for parameter #0 : g1z using max-calls 1500, tolerance 1
******************************************************************************************************
Minuit2Minimizer::GetMinosError - Run MINOS UPPER error for parameter #0 : g1z using max-calls 1500, tolerance 1
Minos: Lower error for parameter g1z  :  -0.00105608
Minos: Upper error for parameter g1z  :  0.00105608
******************************************************************************************************
Minuit2Minimizer::GetMinosError - Run MINOS LOWER error for parameter #1 : ka using max-calls 1500, tolerance 1
******************************************************************************************************
Minuit2Minimizer::GetMinosError - Run MINOS UPPER error for parameter #1 : ka using max-calls 1500, tolerance 1
Minos: Lower error for parameter 

Info in <Minuit2>: MnMinos Determination of lower Minos error for parameter 0
Info in <Minuit2>: MnFunctionCross Run Migrad with fixed parameters:
  Pos 0: g1z = -0.00105608
Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 108.473 μs
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -21.08188278 Edm =   1.172739127e-13 NCalls =      9
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : -21.08188278
  Edm           : 1.172739127e-13
  Internal parameters:	[  0.0007347551333  -0.001455985706]	
  Internal gradient  :	[  9.654981344e-05 -0.0003312305206]	
  Internal covariance matrix:
[[  1.4798472e-05   1.414168e-06]
 [   1.414168e-06  3.8427043e-06]]]
Info in <Minuit2>: VariableMetricBuilder Start iterating until Edm is < 0.0005 with call limit = 1500
Info in <Minuit2>: VariableMetricBuilder    0 - FCN =      -21.08188278 Edm =   1.172739127e-13 NCalls =  

In [15]:
pll0 = nll.createProfile({couplings[0]})
pll1 = nll.createProfile({couplings[1]})
pll2 = nll.createProfile({couplings[2]})

In [16]:
frame0 = couplings[0].frame(Range=(-0.004, 0.004), Title=f";{coupling_names[0]};#Delta NLL")
frame1 = couplings[1].frame(Range=(-0.004, 0.004), Title=f";{coupling_names[1]};#Delta NLL")
frame2 = couplings[2].frame(Range=(-0.004, 0.004), Title=f";{coupling_names[2]};#Delta NLL")
nll.plotOn(frame0, ShiftToZero=True)
nll.plotOn(frame1, ShiftToZero=True)
nll.plotOn(frame2, ShiftToZero=True)

In [17]:
pll0.plotOn(frame0, LineColor="r")
pll1.plotOn(frame1, LineColor="r")
pll2.plotOn(frame2, LineColor="r")

[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[g1z]) Creating instance of MINUIT
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_over_model_Int[O_1,O_2,O_3]_CountingAsimovData0) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[g1z]) determining minimum likelihood for current configurations w.r.t all observable
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[g1z]) minimum found at (g1z=0)
..........................................................................................................................................................................................................
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[ka]) Creating instance of MINUIT
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_over_model_Int[O_1,O_2,O_3]_CountingAsimovData0) Summation contains a RooNLLVar

In [18]:
c0 = ROOT.TCanvas()
frame0.SetMinimum(0)
frame0.SetMaximum(2)
frame0.Draw()
c0.Draw()
# c0.SaveAs("plots/fit/ll_pll.pdf(")
# c0.SaveAs("plots/fit/ll_pll_g1z.pdf")

c1 = ROOT.TCanvas()
frame1.SetMinimum(0)
frame1.SetMaximum(2)
frame1.Draw()
c1.Draw()
# c1.SaveAs("plots/fit/ll_pll.pdf")
# c1.SaveAs("plots/fit/ll_pll_ka.pdf")

c2 = ROOT.TCanvas()
frame2.SetMinimum(0)
frame2.SetMaximum(2)
frame2.Draw()
c2.Draw()
# c2.SaveAs("plots/fit/ll_pll.pdf)")
# c2.SaveAs("plots/fit/ll_pll_la.pdf")